In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime as dt
import warnings
 
warnings.filterwarnings("ignore")
plt.rc('font', family='NanumBarunGothic') 

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# **0. CJ DATA 생성**


In [ ]:
cj = pd.read_excel('./gdrive/My Drive/빅콘 대상팀/data/CJ/cj.xlsx', error_bad_lines = False, parse_dates=['DL_YMD'])
dong = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/지역데이터/전체동별클러스터링.csv', sep = ',', encoding='UTF-8')
dong_cd = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SC/SC동코드변환.csv', sep = ',',encoding='UTF-8')

In [ ]:
cj['CTGG_CD'] = cj['CTGG_CD'].astype('category')
cj['HDNG_CD'] = cj['HDNG_CD'].astype('category')
cj['DL_GD_LCLS_CD'] = cj['DL_GD_LCLS_CD'].astype('category')
cj['CTPV_CD'] = cj['CTPV_CD'].astype('category')
CJ_C = cj

In [ ]:
#cj.groupby(['DL_GD_LCLS_NM']).sum().sort_values(by='INVC_CONT',ascending=False).to_list()
cj_cat_list = ['DL_YMD','식품', '패션의류', '생활건강', '화장품/미용', '디지털/가전', '패션잡화', '도서/음반', '가구/인테리어','스포츠/레저', '출산/육아']
cj = cj.groupby(['DL_YMD','DL_GD_LCLS_NM']).sum().unstack().fillna(0).droplevel(0, axis=1).reset_index(drop=False).reindex(columns=cj_cat_list)
cj.head()
CJ=cj

# **1. 상품 대분류별 일별 물류량**


In [ ]:
cj_a = CJ
x_tick_date = ['2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01', '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01']
x_tick_location = np.array([0,28,59,89,120,149,180,210]) #매월 1일

plt.figure(figsize=(20, 6))

for i in range(len(cj_a.columns)-1):
  plt.plot(cj_a.index, cj_a.iloc[:,i+1],label=cj_a.columns[i+1])

plt.xlim(0,241)
plt.ylim(0,63000)
plt.yticks(fontsize=12)
plt.xticks(x_tick_location,x_tick_date,fontsize=12)
plt.legend(cj_a.columns[1:],loc='lower center',bbox_to_anchor=(0.5, 0.9),ncol=len(cj.columns)-1,fontsize=12)
plt.vlines(x=120,ymin=0, ymax=56000, colors='black', linestyles='--')
plt.title('CJ 일별 물류량',fontsize=20)

# **2. 상품대분류별 주별 물류량 증감률**

In [ ]:
# 설날 2019 2월 4,5,6일 보정
cj = CJ.set_index('DL_YMD')
cj.loc['2019-02-04',:] = (cj.loc['2019-02-11',:] + cj.loc['2019-02-18',:])/2
cj.loc['2019-02-05',:] = (cj.loc['2019-02-12',:] + cj.loc['2019-02-19',:])/2
cj.loc['2019-02-06',:] = (cj.loc['2019-02-13',:] + cj.loc['2019-02-20',:])/2
cj = cj.reset_index(drop=False).query(" DL_YMD not in ['2019-02-01','2020-05-30','2020-05-31']").set_index('DL_YMD',drop=True)
# cj.head()

In [ ]:
# 주차별 데이터 생성
cj_w = cj.resample('W-FRI').mean()
cj_w19 = cj_w['2019-02-01':'2019-05-31']
cj_w20 = cj_w['2020-02-01':'2020-05-31']
cj_w19.index = ['2-1', '2-2', '2-3', '2-4', '3-1', '3-2', '3-3', '3-4', '4-1', '4-2', '4-3', '4-4','4-5','5-1', '5-2', '5-3', '5-4']
cj_w20.index = ['2-1', '2-2', '2-3', '2-4', '3-1', '3-2', '3-3', '3-4', '4-1', '4-2', '4-3', '4-4','4-5','5-1', '5-2', '5-3', '5-4']

# 주차별 성장률
cj_gr = (cj_w20 - cj_w19)/cj_w19*100
cj_gr_cat_list = cj_gr.mean().sort_values(ascending=False).index.to_list()
cj_gr = cj_gr.reindex(columns=cj_gr_cat_list)

In [ ]:
# 전체 데이터 주차별 성장률
def issue_line(min, mycolor):
  plt.axhline(y=0,color='r',linestyle='-',linewidth=2,zorder=9)
  plt.axvline(x='2-3',color=mycolor,linestyle='--')
  plt.text('2-3',min,'신천지', ha='center', va='center',rotation='horizontal', backgroundcolor='white')
  plt.axvline(x='3-2',color=mycolor,linestyle='--')
  plt.text('3-2',min, '마스크 5부제', ha='center', va='center',rotation='horizontal', backgroundcolor='white')
  plt.axvline(x='3-4', color=mycolor,linestyle='--')
  plt.text('3-4',min, '사회적 거리두기', ha='center', va='center',rotation='horizontal', backgroundcolor='white')
  plt.axvline(x='4-2', color=mycolor,linestyle='--')
  plt.text('4-2', min, '고강도 사회적 거리두기', ha='center', va='center',rotation='horizontal', backgroundcolor='white')
  plt.axvline(x='5-1', color=mycolor,linestyle='--')
  plt.text('5-1',min,'생활속 거리두기', ha='center', va='center',rotation='horizontal', backgroundcolor='white')

# 전체 데이터 주차별 성장률 그래프
plt.figure(figsize=(20, 6))
my_colors = ['#1f77b4', '#2ca02c'] #['#9467bd',  '#e377c2']#, ['#1f77b4', '#2ca02c'],'#ff7f0e
for i in range(len(list(cj_gr.columns)[:2])):
  plt.plot(cj_gr[list(cj_gr.columns)[i]],my_colors[i],linewidth=3,zorder=i+8)
for j in range(len(list(cj_gr.columns)[2:])):
  plt.plot(cj_gr[list(cj_gr.columns)[j+2]],color='silver',zorder=i-2)
plt.legend(cj_gr.columns)
plt.xlim('2-1','5-4')
plt.ylim(-30,190)
plt.yticks(fontsize=12)
# plt.ylabel('증감률(%)')
plt.legend(cj_gr.columns,loc='center',bbox_to_anchor=(0.5, 0.95),ncol=len(cj_gr.columns),fontsize=12)
min = cj_gr.values.min()
issue_line(min,'grey')

plt.title('CJ 물류 주별 증감률(%)',fontsize=20)

# **3. 클러스터별 주별 물류량 증감률**

In [ ]:
cj_c = CJ_C
cj_c = pd.merge(cj_c, dong, left_on='HDNG_NM', right_on='HDONG_NM').query("SIDO_NM=='서울'")
cj_c = cj_c .replace(np.nan, 0)
cj_c = cj_c.groupby(['DL_YMD','DL_GD_LCLS_NM','CLUS_NM']).sum().unstack()
cj_c = cj_c.unstack().droplevel(0,axis=1).fillna(0)

for i in ['노원구 대학가', '노원구 학원가','중구 빌딩숲', '중구 소상공']:
  cj_c.xs(i,axis=1,level=0).loc['2019-02-04',:] = (cj_c.xs(i,axis=1,level=0).loc['2019-02-11',:] + cj_c.xs(i,axis=1,level=0).loc['2019-02-18',:])/2
  cj_c.xs(i,axis=1,level=0).loc['2019-02-05',:] = (cj_c.xs(i,axis=1,level=0).loc['2019-02-12',:] + cj_c.xs(i,axis=1,level=0).loc['2019-02-19',:])/2
  cj_c.xs(i,axis=1,level=0).loc['2019-02-06',:] = (cj_c.xs(i,axis=1,level=0).loc['2019-02-13',:] + cj_c.xs(i,axis=1,level=0).loc['2019-02-20',:])/2

cj_c = cj_c.reset_index(drop=False)
cj_c = cj_c[cj_c.DL_YMD != '2019-02-01'] ; cj_c = cj_c[cj_c.DL_YMD != '2020-05-30'] ; cj_c = cj_c[cj_c.DL_YMD != '2020-05-31'] 
cj_c = cj_c.set_index('DL_YMD').stack()

In [ ]:
cj_c19 = cj_c.unstack().loc['2019-02-01':'2019-05-31',:]
cj_c20 = cj_c.unstack().loc['2020-02-01':'2020-05-31',:]
cj_c19 = cj_c19.resample('W-FRI').mean()
cj_c20 = cj_c20.resample('W-FRI').mean()
cj_c19.index = ['2-1', '2-2', '2-3', '2-4', '3-1', '3-2', '3-3', '3-4', '4-1', '4-2', '4-3', '4-4','4-5','5-1', '5-2', '5-3', '5-4']
cj_c20.index = ['2-1', '2-2', '2-3', '2-4', '3-1', '3-2', '3-3', '3-4', '4-1', '4-2', '4-3', '4-4','4-5','5-1', '5-2', '5-3', '5-4']
cj_c_gr = (cj_c20 - cj_c19)/cj_c19 *100
# cj_c_gr = cj_c_gr.stack().droplevel(0,axis=1)
cj_c_gr = cj_c_gr.T.reindex(cj_gr_cat_list,level=1)

In [ ]:
def issue_line_2(min, mycolor):
  plt.axhline(y=0,color='r',linestyle='-',linewidth=2,zorder=9)
  plt.axvline(x='2-3',color=mycolor,linestyle='--')
  plt.text('2-3',min,'신천지', ha='center', va='center',rotation='horizontal', backgroundcolor='white')
  plt.axvline(x='3-2',color=mycolor,linestyle='--')
  plt.text('3-2',min, '마스크 5부제', ha='center', va='center',rotation='horizontal', backgroundcolor='white')
  plt.axvline(x='3-4', color=mycolor,linestyle='--')
  plt.text('3-4',min+10, '사회적 거리두기', ha='center', va='center',rotation='horizontal', backgroundcolor='white')
  plt.axvline(x='4-2', color=mycolor,linestyle='--')
  plt.text('4-2', min, '고강도 사회적 거리두기', ha='center', va='center',rotation='horizontal', backgroundcolor='white')
  plt.axvline(x='5-1', color=mycolor,linestyle='--')
  plt.text('5-1',min,'생활속 거리두기', ha='center', va='center',rotation='horizontal', backgroundcolor='white')


In [ ]:
# 노원구 대학가 주차별 성장률 그래프
plt.figure(figsize=(8, 6))
my_colors = ['#1f77b4', '#2ca02c','#9467bd',  '#e377c2'] # ['#1f77b4', '#2ca02c'],'#ff7f0e
for i in range(len(list(cj_gr.columns)[:4])):
  plt.plot(cj_c_gr.xs('노원구 대학가',axis=0,level=0).T[list(cj_gr.columns)[i]],my_colors[i],linewidth=3,zorder=i+6)
for j in range(len(list(cj_gr.columns)[4:])):
  plt.plot(cj_c_gr.xs('노원구 대학가',axis=0,level=0).T[list(cj_gr.columns)[j+4]],color='silver',zorder=i-4)

plt.xlim('2-1','5-4')
plt.ylim(-50,140)
# plt.legend(cj_gr.columns,loc='center',bbox_to_anchor=(0.5, 0.95),ncol=len(cj_gr.columns),fontsize=12)
plt.legend(cj_gr.columns[:4],loc='center',bbox_to_anchor=(0.89, 0.895),ncol=1,fontsize=10,framealpha=1)
min = -40
issue_line_2(min,'grey')
plt.title('CJ 물류 노원구 대학가 주차별 증감률',fontsize=20)

In [ ]:
# 노원구 학원가 주차별 성장률 그래프
plt.figure(figsize=(8, 6))
my_colors = ['#1f77b4', '#2ca02c','#9467bd',  '#e377c2'] # ['#1f77b4', '#2ca02c'],'#ff7f0e
for i in range(len(list(cj_gr.columns)[:4])):
  plt.plot(cj_c_gr.xs('노원구 학원가',axis=0,level=0).T[list(cj_gr.columns)[i]],my_colors[i],linewidth=3,zorder=i+6)
for j in range(len(list(cj_gr.columns)[4:])):
  plt.plot(cj_c_gr.xs('노원구 학원가',axis=0,level=0).T[list(cj_gr.columns)[j+4]],color='silver',zorder=i-4)

plt.xlim('2-1','5-4')
plt.ylim(-50,140)
# plt.legend(cj_gr.columns,loc='center',bbox_to_anchor=(0.5, 0.95),ncol=len(cj_gr.columns),fontsize=12)
plt.legend(cj_gr.columns[:4],loc='center',bbox_to_anchor=(0.89, 0.895),ncol=1,fontsize=10,framealpha=1)
min = -40
issue_line_2(min,'grey')

plt.title('CJ 물류 노원구 학원가 주차별 증감률',fontsize=20)

In [ ]:
# 중구 빌딩숲 주차별 성장률 그래프
plt.figure(figsize=(8, 6))
my_colors = ['#1f77b4', '#2ca02c','#9467bd',  '#e377c2'] 
for i in range(len(list(cj_gr.columns)[:4])):
  plt.plot(cj_c_gr.xs('중구 빌딩숲',axis=0,level=0).T[list(cj_gr.columns)[i]],my_colors[i],linewidth=3,zorder=i+6)
plt.plot(cj_c_gr.xs('중구 빌딩숲',axis=0,level=0).T[list(cj_gr.columns)[8]],color='crimson',linestyle='--',zorder=4)
for j in range(len(list(cj_gr.columns)[4:8])):
  plt.plot(cj_c_gr.xs('중구 빌딩숲',axis=0,level=0).T[list(cj_gr.columns)[j+4]],color='silver',zorder=i-4)

plt.plot(cj_c_gr.xs('중구 빌딩숲',axis=0,level=0).T[list(cj_gr.columns)[9]],color='silver',zorder=5)
plt.xlim('2-1','5-4')
plt.ylim(-90,100)
# plt.legend(cj_gr.columns,loc='center',bbox_to_anchor=(0.5, 0.95),ncol=len(cj_gr.columns),fontsize=12)
plt.legend(['식품','생활건강','도서/음반','가구/인테리어','출산/육아'],loc='center',bbox_to_anchor=(0.89, 0.87),ncol=1,fontsize=10,framealpha=1)
min = -83
issue_line_2(min,'grey')

plt.title('CJ 물류 중구 빌딩숲 주별 증감률',fontsize=20)

In [ ]:
# 중구 소상공 주차별 성장률 그래프
plt.figure(figsize=(8, 6))
my_colors = ['#1f77b4', '#2ca02c','#9467bd',  '#e377c2'] 
for i in range(len(list(cj_gr.columns)[:4])):
  plt.plot(cj_c_gr.xs('중구 소상공',axis=0,level=0).T[list(cj_gr.columns)[i]],my_colors[i],linewidth=3,zorder=i+6)
for j in range(len(list(cj_gr.columns)[4:])):
  plt.plot(cj_c_gr.xs('중구 소상공',axis=0,level=0).T[list(cj_gr.columns)[j+4]],color='silver',zorder=i-4)

plt.xlim('2-1','5-4')
plt.ylim(-65,125)
# plt.legend(cj_gr.columns,loc='center',bbox_to_anchor=(0.5, 0.95),ncol=len(cj_gr.columns),fontsize=12)
plt.legend(cj_gr.columns[:4],loc='center',bbox_to_anchor=(0.89, 0.895),ncol=1,fontsize=10,framealpha=1)
min = -58
issue_line_2(min,'grey')

plt.title('CJ 물류 중구 소상공 주별 증감률',fontsize=20)

In [ ]:
print('[클러스터별 평균 증감률]')
print('중구 빌딩숲   :',cj_c_gr.xs('중구 빌딩숲',axis=0,level=0).mean().mean())
print('중구 소상공   :',cj_c_gr.xs('중구 소상공',axis=0,level=0).mean().mean())
print('노원구 학원가 :',cj_c_gr.xs('노원구 학원가',axis=0,level=0).mean().mean())
print('노원구 대학가 :',cj_c_gr.xs('노원구 대학가',axis=0,level=0).mean().mean())